In [9]:
from utils.data_preprocess import *
from utils.tf_helpers import create_model_checkpoint, evaluate_preds
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
df = DataPreprocess('ETHBTC-5m-data.csv').get_ohlcv()

In [10]:
X, y = make_windows(df, 96)

In [11]:
X.shape

(453489, 100)

In [12]:
from tensorflow.keras import Input, layers, Model, Sequential

# Create CNN blocks for feature extraction

def CNN_blocks(filters, kernel_size, pool_size, pool_strides):
    block = Sequential([
        layers.Lambda(lambda x: tf.expand_dims(x, axis=1)),
        layers.Convolution2D(filters=filters, kernel_size=kernel_size, strides=1),
        layers.Lambda(lambda x: tf.squeeze(x, axis=1)),
        layers.MaxPool1D(pool_size=pool_size, strides=pool_strides)])
    
    return block
    


In [15]:
# Create model using Model API
input = Input(shape=(1, 100), name='input_layer')
x = CNN_blocks(filters=32, kernel_size=(4, 32), pool_size=4, pool_strides=4)(input)
x = CNN_blocks(filters=64, kernel_size=(3, 64), pool_size=3, pool_strides=3)(x)
x = CNN_blocks(filters=128, kernel_size=(2, 128), pool_size=2, pool_strides=2)(x)
x = layers.Dense(1000, activation='relu')(x)
x = layers.Dense(500, activation='relu')(x)
output = layers.Dense(1, activation='linear')(x)
model = Model(input, output, name='strong_baseline')

# Compile the model
model.compile(optimizer='adam', loss='mae', metrics=['mae'])

ValueError: Negative dimension size caused by subtracting 4 from 1 for '{{node sequential_2/conv2d_2/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](sequential_2/lambda_4/ExpandDims, sequential_2/conv2d_2/Conv2D/ReadVariableOp)' with input shapes: [?,1,1,100], [4,32,100,32].